<a href="https://colab.research.google.com/github/pradeepsabic/NLP/blob/main/Langchain_Vectordb_PDFLoad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
pip install uvicorn

In [ ]:
#pip install pypdf,langchain,langchain-openai,langchain-community,fastapi
#for web access from colab uvicorn pyngrok

This will extract all the text from the PDF file and store it in the text variable.

In [14]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings  # Or other embeddings like Hugging Face
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Open the PDF file using LangChain's PyPDFLoader
pdf_path = 'Pradeep-Summary.pdf'
loader = PyPDFLoader(pdf_path)
# Load the documents (pages) from the PDF
documents = loader.load()
# Extract text from each document (page)
text = ""
for doc in documents:
    text += doc.page_content

# Print or use the extracted text
print(text)

**Pradeep Kumar Sharma – IT Professional, Cloud Computing, NLP, and Deep Learning Expert**  
 
Pradeep Kumar Sharma is a dynamic and versatile IT professional with over 17 years of extensive 
experience in the fields of Cloud computing, Natural Language Processing (NLP), and Deep Learning. 
His journey in the tech industry has seen him work with prest igious global organizations like COLT, 
SDS, and SABIC, where he honed his expertise in cutting -edge Microsoft technologies, Azure DevOps, 
and cloud infrastructures. Throughout his career, Pradeep has built a reputation for driving 
innovation, delivering re sults, and transforming complex business challenges into scalable and 
efficient solutions.  
 
### **Professional Expertise and Experience**  
 
#### Cloud Computing and Azure DevOps Mastery  
Pradeep has been at the forefront of the cloud computing revolution, wo rking with Microsoft Azure, 
one of the leading cloud platforms, to develop and manage complex cloud infrastructures. H

Clearning and preporcessing of the text

In [8]:
import re
# Remove extra whitespace and new lines
text = re.sub(r'\s+', ' ', text)
# Remove numbers
#text = re.sub(r'\d+', '', text)
# Remove punctuation
text = re.sub(r'[^\w\s]', '', text)
# Remove ###, **, and -
text = re.sub(r'[#\*\-]+', '', text)
# Remove leading/trailing whitespace
text = text.strip()

print(text)

Pradeep Kumar Sharma  IT Professional Cloud Computing NLP and Deep Learning Expert Pradeep Kumar Sharma is a dynamic and versatile IT professional with over 17 years of extensive experience in the fields of Cloud computing Natural Language Processing NLP and Deep Learning His journey in the tech industry has seen him work with prest igious global organizations like COLT SDS and SABIC where he honed his expertise in cutting edge Microsoft technologies Azure DevOps and cloud infrastructures Throughout his career Pradeep has built a reputation for driving innovation delivering re sults and transforming complex business challenges into scalable and efficient solutions  Professional Expertise and Experience  Cloud Computing and Azure DevOps Mastery Pradeep has been at the forefront of the cloud computing revolution wo rking with Microsoft Azure one of the leading cloud platforms to develop and manage complex cloud infrastructures His in  depth knowledge of cloud architecture automation and 

**Split Text into Chunks**

In [15]:
# Split the text into smaller chunks for embedding
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=5)
split_docs = text_splitter.split_documents(documents)


In [20]:
import os
from google.colab import drive
drive.mount('/content/drive')
with open('/content/drive/MyDrive/openai_key.txt', 'r') as file:
    openai_api_key = file.read().strip()
    os.environ["OPENAI_API_KEY"] = openai_api_key

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Embedding Model to Convert Text into Vectors**

In [23]:
embedding = OpenAIEmbeddings()

**Store the Vectors in ChromaDB**

In [24]:
# Create or connect to a ChromaDB vector store
vector_store = Chroma.from_documents(documents=split_docs, embedding=embedding)

# Save the vector store for later use
vector_store.persist()

<ipython-input-24-0f3645b912f9>:5: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


Query the Vector DB

In [25]:
query = "Tell me about Pradeep Kumar Sharma's professional background"
results = vector_store.similarity_search(query)

# Display results
for result in results:
    print(result.page_content)

Pradeep Kumar Sharma stands out as a leader in th e IT, Cloud, NLP, and Deep Learning fields, with a 
proven track record of delivering successful projects and driving business transformation. His 
passion for innovation, combined with his technical expertise and leadership abilities, has made him 
an invalu able asset to every organization he has worked with. As the tech industry continues to 
evolve, Pradeep remains committed to staying at the cutting edge, ensuring that the solutions he 
delivers not only meet current demands but also anticipate future needs.
**Pradeep Kumar Sharma – IT Professional, Cloud Computing, NLP, and Deep Learning Expert**  
 
Pradeep Kumar Sharma is a dynamic and versatile IT professional with over 17 years of extensive 
experience in the fields of Cloud computing, Natural Language Processing (NLP), and Deep Learning. 
His journey in the tech industry has seen him work with prest igious global organizations like COLT, 
SDS, and SABIC, where he honed his ex

In [26]:
vector_store.persist()

**Build an API Using Flask or FastAPI**

In [28]:
from fastapi import FastAPI, UploadFile, File

In [29]:
app = FastAPI()

In [31]:
@app.post("/query/")
async def query_text(query: str):
    # Perform the query directly using your model or vector store
    if vector_store is None:
        return {"message": "No documents have been uploaded yet."}

    # Search the vector store for relevant documents based on the query
    results = vector_store.similarity_search(query)

    # Return the most relevant document contents
    return {"results": [result.page_content for result in results]}